In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib as ta # This is the main TaLib import!
import os

ModuleNotFoundError: No module named 'talib'

In [ ]:
# If notebook is in 'notebooks/', data is in '../data/'
data_file_path = '../data/stock_prices.csv'

# Check if data directory exists, create if not
if not os.path.exists('../data'):
    os.makedirs('../data')
    print("Created 'data/' directory.")

df = pd.DataFrame() # Initialize df to avoid NameError

if os.path.exists(data_file_path):
    print(f"Loading data from: {data_file_path}")
    try:
        df = pd.read_csv(data_file_path, index_col='Date', parse_dates=True)
        print("Data loaded successfully.")
    except Exception as e:
        print(f"Error loading CSV: {e}. Generating dummy data instead.")
        # Fallback to dummy data if CSV fails
        dates = pd.date_range(start='2023-01-01', periods=252, freq='B') # 252 trading days
        df = pd.DataFrame({
            'Open': np.random.uniform(100, 110, len(dates)),
            'High': np.random.uniform(105, 115, len(dates)),
            'Low': np.random.uniform(95, 105, len(dates)),
            'Close': np.random.uniform(100, 110, len(dates)),
            'Volume': np.random.randint(1_000_000, 5_000_000, len(dates))
        }, index=dates)
        print("Using generated dummy data.")
else:
    print(f"Data file not found at {data_file_path}. Generating dummy data.")
    # Generate dummy data
    dates = pd.date_range(start='2023-01-01', periods=252, freq='B') # 252 trading days
    df = pd.DataFrame({
        'Open': np.random.uniform(100, 110, len(dates)),
        'High': np.random.uniform(105, 115, len(dates)),
        'Low': np.random.uniform(95, 105, len(dates)),
        'Close': np.random.uniform(100, 110, len(dates)),
        'Volume': np.random.randint(1_000_000, 5_000_000, len(dates))
    }, index=dates)
    print("Generated dummy data.")

In [ ]:
# Ensure 'Close' column is numeric and handle NaNs
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')
df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
df['High'] = pd.to_numeric(df['High'], errors='coerce')
df['Low'] = pd.to_numeric(df['Low'], errors='coerce')
df.dropna(subset=['Close', 'Open', 'High', 'Low'], inplace=True)
# Display first few rows and info
print("\nDataFrame Head:")
print(df.head())
print("\nDataFrame Info:")
df.info()

In [ ]:
# Convert necessary columns to numpy arrays
close_prices = df['Close'].values
open_prices = df['Open'].values
high_prices = df['High'].values
low_prices = df['Low'].values

# Calculate Simple Moving Average (SMA)
# Common period for short-term SMA is 20, 50, or 200
df['SMA_20'] = ta.SMA(close_prices, timeperiod=20)
print("\nSMA (20) calculated.")

# Calculate Relative Strength Index (RSI)
# Common period for RSI is 14
df['RSI_14'] = ta.RSI(close_prices, timeperiod=14)
print("RSI (14) calculated.")

# Calculate Moving Average Convergence Divergence (MACD)
# Default periods: fastperiod=12, slowperiod=26, signalperiod=9
macd, macdsignal, macdhist = ta.MACD(
    close_prices,
    fastperiod=12,
    slowperiod=26,
    signalperiod=9
)
df['MACD'] = macd
df['MACD_Signal'] = macdsignal
df['MACD_Hist'] = macdhist # Histogram is the difference between MACD and Signal
print("MACD, MACD Signal, and MACD Histogram calculated.")

# Display the tail of the DataFrame to see the new columns
print("\nDataFrame Tail with Indicators:")
print(df.tail())

In [ ]:
plt.style.use('seaborn-v0_8-darkgrid') # A nice style for plots

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 10), sharex=True,
                                    gridspec_kw={'height_ratios': [3, 1.5, 2]}) # Adjust subplot heights

# --- Plot 1: Close Price and SMA ---
ax1.plot(df.index, df['Close'], label='Close Price', color='blue', linewidth=1.5)
ax1.plot(df.index, df['SMA_20'], label='SMA (20)', color='orange', linestyle='--', linewidth=1.5)
ax1.set_title('Stock Price and Simple Moving Average (SMA)')
ax1.set_ylabel('Price')
ax1.legend()
ax1.grid(True)

# --- Plot 2: RSI ---
ax2.plot(df.index, df['RSI_14'], label='RSI (14)', color='purple', linewidth=1.5)
ax2.axhline(70, linestyle='--', alpha=0.6, color='red', label='Overbought (70)')
ax2.axhline(30, linestyle='--', alpha=0.6, color='green', label='Oversold (30)')
ax2.set_title('Relative Strength Index (RSI)')
ax2.set_ylabel('RSI Value')
ax2.set_ylim(0, 100) # RSI is always between 0 and 100
ax2.legend()
ax2.grid(True)

# --- Plot 3: MACD ---
ax3.plot(df.index, df['MACD'], label='MACD Line', color='green', linewidth=1.5)
ax3.plot(df.index, df['MACD_Signal'], label='Signal Line', color='red', linestyle='--', linewidth=1.5)
# Plot MACD Histogram as bars
colors = ['g' if val >= 0 else 'r' for val in df['MACD_Hist']]
ax3.bar(df.index, df['MACD_Hist'], label='MACD Histogram', color=colors, alpha=0.5, width=pd.Timedelta(days=5)) # Adjust width for daily data
ax3.set_title('Moving Average Convergence Divergence (MACD)')
ax3.set_xlabel('Date')
ax3.set_ylabel('MACD Value')
ax3.axhline(0, color='gray', linestyle='-', linewidth=0.8) # Zero line for MACD
ax3.legend()
ax3.grid(True)

plt.tight_layout() # Adjusts subplot params for a tight layout
plt.show()